In [1]:
import logging
import numpy as np
import pandas as pd
import h5py as h5
from collections import namedtuple

In [3]:
logging.basicConfig(level=1, format='[%(asctime)s %(levelname)s] %(message)s')
Option = namedtuple("MyStruct", "input train_size")
                    # model name folds batch_size epochs")
                    #structure learning_rate regularizer epochs")
args = Option(
    input='data/total.h5',
    train_size = 0.75
    #model='share/reference_number.py',
    #name='NumberNetworkKFOLD',
    #folds=10,
    #batch_size=60,
    #epochs=20,
    #learning_rate=0.0001,
    #regularizer=0.0001,
)

# Load Data

Load from one file

In [6]:
logging.info(f'Loading data from {args.input}')

with h5.File(args.input, 'r') as data:
    data_x = data['input'][()]
    data_y = data['target'][()]



NameError: name 'args' is not defined

Load from endcap, barrel, ibl files

In [2]:
with h5.File('data/oldjz7wendcapwithflip.h5', 'r') as data:
    data_x = data['input'][:int(5e6)]
    data_y = data['target'][:int(5e6)]
with h5.File('data/oldjz7wiblwithflip.h5', 'r') as data:
    data_x = np.append(data_x, data['input'][:int(5e6)], axis=0)
    data_y = np.append(data_y, data['target'][:int(5e6)], axis=0)
with h5.File('data/oldjz7wbarrelwithflip.h5', 'r') as data:
    data_x = np.append(data_x, data['input'][:int(5e6)], axis=0)
    data_y = np.append(data_y, data['target'][:int(5e6)], axis=0)

In [3]:
data_x.shape

(15000000, 60)

# Shuffle Data

In [5]:
logging.info('Shuffling')
rng = np.random.default_rng(seed=42)
s = np.arange(data_x.shape[0])
rng.shuffle(s)

data_x = data_x[s]
data_y = data_y[s]

with h5.File(f'data/total_shuffled.h5', 'w') as hfile:
    hfile.create_dataset('input', data=data_x.astype(np.float32), compression='gzip')
    hfile.create_dataset('target', data=data_y.astype(np.float32), compression='gzip')

In [11]:
train_split = range(int(data_x.shape[0]*args.train_size))
train_split

range(0, 12750000)

In [12]:
test_split = range(int(data_x.shape[0]*args.train_size), data_x.shape[0])
test_split

range(12750000, 17000000)

In [13]:
data_x[train_split].shape

(12750000, 60)

In [14]:
data_x[test_split].shape

(4250000, 60)

# Create Train and Test splits

In [15]:
with h5.File(f'data/train.h5', 'w') as hfile:
    hfile.create_dataset('input', data=data_x[train_split].astype(np.float32), compression='gzip')
    hfile.create_dataset('target', data=data_y[train_split].astype(np.float32), compression='gzip')

with h5.File('data/test.h5', 'w') as hfile:
    hfile.create_dataset('input', data=data_x[test_split].astype(np.float32), compression='gzip')
    hfile.create_dataset('target', data=data_y[test_split].astype(np.float32), compression='gzip')

# Create 10 Splits

In [20]:
for i in range(10):
    with h5.File(f'data/split{i+1}.h5', 'w') as hfile:
        hfile.create_dataset('input', data=data_x[int(data_x.shape[0]/10)*i:int(data_x.shape[0]/10)*(i+1)].astype(np.float32), compression='gzip')
        hfile.create_dataset('target', data=data_y[int(data_x.shape[0]/10)*i:int(data_x.shape[0]/10)*(i+1)].astype(np.float32), compression='gzip')
